<a href="https://colab.research.google.com/github/thomasjithin440/thomasjithin440/blob/main/Action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pyfluidsynth
!fluidsynth --version


/bin/bash: line 1: fluidsynth: command not found


In [6]:
# Text-to-Music Generation Project

## Step 1: Install Dependencies
# Install necessary libraries. Run this cell if you haven’t installed the libraries already.
#!pip install transformers torch pretty_midi librosa soundfile pyfluidsynth

## Step 2: Import Libraries
import torch
from transformers import AutoTokenizer, AutoModel
import pretty_midi
import fluidsynth
import soundfile as sf
from IPython.display import Audio

# Define text to music functions

## Step 3: Text Embedding with NLP Model
def get_text_embeddings(input_text):
    """
    This function takes an input text and generates embeddings using a pretrained language model.
    """
    # Load a pretrained model and tokenizer
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Tokenize input text
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate embeddings
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)

    return embeddings

## Step 4: Generate MIDI from Embeddings
def generate_midi(embeddings, output_file="output.mid"):
    """
    Generate MIDI file from text embeddings.

    Args:
        embeddings (torch.Tensor): Input embeddings generated from the text.
        output_file (str): File name for the generated MIDI file.

    Returns:
        None (saves a MIDI file to disk)
    """
    midi = pretty_midi.PrettyMIDI()
    piano_program = pretty_midi.instrument_name_to_program("Acoustic Grand Piano")
    piano = pretty_midi.Instrument(program=piano_program)

    # Generate notes based on embeddings
    for i, embedding in enumerate(embeddings[0]):
        note_number = int((embedding.item() + 1) * 36)  # Scale to MIDI note range (0-127)
        start_time = i * 0.5
        end_time = start_time + 0.5

        # Add note to the instrument
        note = pretty_midi.Note(velocity=100, pitch=max(0, min(127, note_number)), start=start_time, end=end_time)
        piano.notes.append(note)

    midi.instruments.append(piano)
    midi.write(output_file)

## Step 5: Convert MIDI to Audio
def midi_to_audio(midi_file, output_audio="output.wav", soundfont="default.sf2"):
    """
    Convert a MIDI file to an audio file using fluidsynth.

    Args:
        midi_file (str): Path to the input MIDI file.
        output_audio (str): Path to save the output audio.
        soundfont (str): Path to a .sf2 soundfont file.

    Returns:
        None (saves an audio file to disk)
    """
    # Use a default soundfont if none provided
    if soundfont == "default.sf2":
        !wget -O default.sf2 https://freepats.zenvoid.org/freepats-general-midi.sf2
        soundfont = "default.sf2"

    # Initialize FluidSynth with the soundfont
    fs = fluidsynth.Synth()
    fs.start(driver="alsa")
    fs.sfload(soundfont)

    # Convert MIDI to audio
    fs.midi_to_audio(midi_file, output_audio)
    fs.delete()

## Step 6: Pipeline Function
def text_to_music():
    """
    Complete pipeline for text-to-music generation.

    Returns:
        Audio playback of the generated music.
    """
    input_text = input("Enter a description for the music you'd like (e.g., 'Calm and relaxing piano melody'): ")
    print("Generating text embeddings...")
    embeddings = get_text_embeddings(input_text)

    print("Generating MIDI file...")
    generate_midi(embeddings, "generated_music.mid")

    print("Converting MIDI to audio...")
    midi_to_audio("generated_music.mid", "generated_music.wav")

    print("Done! Playing audio...")
    return Audio("generated_music.wav", autoplay=True)

## Step 7: Run the Pipeline
output_audio = text_to_music()
output_audio


ImportError: Couldn't find the FluidSynth library.